In [3]:
# Data generation

import numpy as np
from itertools import permutations
import random

Nexam = 10**5
Ndim = 19
Nperm = 30
x1 = np.random.multivariate_normal(mean= np.ones(Ndim), cov = np.identity(Ndim),size = Nexam)
y1 = np.ones(Nexam)
x2 = np.random.multivariate_normal(mean= -np.ones(Ndim), cov = np.identity(Ndim),size = Nexam)
y2 = -np.ones(Nexam)

X = np.concatenate((x1,x2),axis=0)
y = np.concatenate((y1,y2))


from sklearn import model_selection
X_tr_orig, X_test_orig, y_tr_orig, y_test_orig = model_selection.train_test_split(X,y,test_size=0.5)

# reshaping y
y_tr_orig = y_tr_orig.reshape((len(y_tr_orig),1))
y_test_orig = y_test_orig.reshape((len(y_test_orig),1))

perms = [] # array storing different premutatins of X, Y
for _ in range(Nperm):
    inx = np.random.permutation(Nexam)
    X_perm = X_tr_orig[inx]
    y_perm = y_tr_orig[inx]
    perms.append([X_perm,y_perm])

### a) Batch newton algorithm with the Gauss-Newton approximation

In [4]:
# training batch newton algo
import math
from numpy.linalg import multi_dot
from numpy import outer, matmul
from numpy.linalg import inv, norm
import time
Nsizes = 3
# n_ex = np.floor(np.logspace(2.0, 5.0, num=Nsizes))
n_ex = [1000]

start_time = time.time()
theta = np.random.uniform(-0.5, 0.5, size=Ndim).reshape((Ndim,1))

theta_store2 = []
for N in n_ex:
    theta_store1 = []
    thresh = 1.0/N
    perm_counter = 0
    for X_tr,y_tr in perms: 
        perm_counter += 1
        print "perm counter:", perm_counter 
#         g = np.zeros(Ndim)
        h = np.zeros((Ndim,Ndim))
        keep_iter = True
        counter = 0
        while keep_iter:
            counter += 1
            
            if counter % 10 == 0:
                print "iter:", counter
                print "error", norm(d_theta)
                print "threshold", thresh
            
            f = 1.71 * np.tanh(0.66 * matmul(X_tr[:N,:], theta))
            df = 1.71 * 0.66 * X_tr[:N,:] / ( np.cosh(0.66 * matmul(X_tr[:N,:], theta)) )**2
            g_mat = 2 * np.diag(f - 1.5*y_tr[:N]) * df
            g = np.sum(g_mat,axis=0).reshape((Ndim,1))
            for i in range(N):
                h += matmul(np.outer(np.transpose(df[i,:]),df[i,:]), np.outer(np.transpose(X_tr[i,:]),X_tr[i,:]))
            h_inv = inv(h)
            d_theta = -matmul(h_inv,g)
            if norm(d_theta) > thresh:
                theta += d_theta
            else:
                keep_iter = False
        theta_store1.append(theta)
    theta_store2.append(theta_store1)
end_time = time.time()
elapsed_time = end_time - start_time
            
            

perm counter: 1
iter: 10
error 0.00282752942954
threshold 0.001
iter: 20
error 0.00133625640494
threshold 0.001
perm counter: 2
iter: 10
error 0.0022332821514
threshold 0.001
iter: 20
error 0.00107207888061
threshold 0.001
perm counter: 3
iter: 10
error 0.00112148684602
threshold 0.001
perm counter: 4
iter: 10
error 0.00260234176184
threshold 0.001
iter: 20
error 0.00123444564744
threshold 0.001
perm counter: 5
perm counter: 6
iter: 10
error 0.00480247465559
threshold 0.001
iter: 20
error 0.00231839975391
threshold 0.001
iter: 30
error 0.00153688889175
threshold 0.001
iter: 40
error 0.00115261997559
threshold 0.001
perm counter: 7
iter: 10
error 0.00262366391655
threshold 0.001
iter: 20
error 0.00126038920029
threshold 0.001
perm counter: 8
iter: 10
error 0.00183562007375
threshold 0.001
perm counter: 9
iter: 10
error 0.00400787006289
threshold 0.001
iter: 20
error 0.0019342622986
threshold 0.001
iter: 30
error 0.00128168323258
threshold 0.001
perm counter: 10
iter: 10
error 0.00354102

In [7]:
# evaluation of newton method on test set
from sklearn.metrics import mean_squared_error
store_mse = []
X_test = X_test_orig
y_test = y_test_orig
for i, N in enumerate(n_ex):
    mse = 0
    for j in range(Nperm):
        theta = theta_store2[i][j]
        f = 1.71 * np.tanh(0.66 * matmul(X_test, theta))
        mse += mean_squared_error(f,y_test)/Nperm
    store_mse.append([mse,N])

In [8]:
store_mse

[[3.3340526864529134, 1000]]